## 학습목표
 1. datagrame group by 
 2. concat, merge, pivot, pivot_table, stack, unstack, transorm 등을 통한 여러가지 dataframe 변형

### group by
  + 아래의 세 단계를 포함 (SQL의 group by 와 개념적으로는 동일, 사용법은 유사)
    - 데이터 분할
    - operation 적용
    - 데이터 병합

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 1955년부터의 유로리그 우승팀 데이터
df = pd.read_csv('./data/euro_winners.csv')

In [ ]:
df.head(15)

In [ ]:
# 국가별로 그룹핑 
nation_group = df.groupby('Nation')
type(nation_group)

In [ ]:
print dir(nation_group)

In [ ]:
nation_group.groups # groups 속성으로 group 정보를 dict로 반환

In [ ]:
len(nation_group.groups) # 그룹의 개수

* group 함수
 - count, size 등

In [ ]:
nation_group.count()

In [ ]:
nation_group.size()

In [ ]:
nation_group.size().sort_values(ascending = False)

In [ ]:
# 2개 이상의 컬럼으로 grouping 가능
club_group = df.groupby(['Nation', 'Winners'])
club_group.size().sort_values(ascending = False)

In [ ]:
df = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [ ]:
df.head(20)

In [ ]:
# Month column을 index로 생성
#df = df.set_index('Month')

# 혹은 아래와 같이 가능
df.set_index(['Month', 'Stat'], inplace=True)

In [ ]:
df.head()

In [ ]:
# 다시 index를 column 레벨로 변경
df = df.reset_index()

In [ ]:
df.head(100)

In [ ]:
# group by year
# 함수가 전달되는 경우 함수의 파라미터로 index 값이 전달
year_group = df.groupby(lambda month : month.split('/')[2])

In [ ]:
year_group.groups

In [ ]:
year_group.size()

In [ ]:
for name, group in year_group:
    print name
    print group

In [ ]:
# index의 경우에는 column name이 아닌, 레벨로 grouping 가능
month_group = df.groupby(level = 0)

In [ ]:
# 혹은 다음과 같이 index 자체를 명시 가능
month_group = df.groupby(level = 'Month')

In [ ]:
for name, group in month_group:
    print name
    print group
    print '*'

### multi index grouping

* index 항목 제거

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

* 2가지 컬럼으로 인덱싱

In [ ]:
# 멀티 레벨 인덱스를 생성
df = df.set_index(['Month', 'Stat'])
df.head()

* 멀티 인덱스에서 row 선택

In [ ]:
df.loc['08/01/2012'].loc['MatchesPlayed']

In [ ]:
df.loc['08/01/2012', 'MatchesPlayed']

* 멀티 인덱스의 레벨을 이용한 그룹핑

In [ ]:
# multi level grouping
month_stat_group = df.groupby(level = [0, 1])

In [ ]:
for name, group in month_stat_group:
    print name # tuple
    print group

In [ ]:
month_stat_group = df.groupby(level = ['Month', 'Stat'])

In [ ]:
for name, group in month_stat_group:
    print name # tuple
    print group

* row 선택하기

In [ ]:
df2 = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [ ]:
df2.head()

In [ ]:
df2.loc[0] # 0은 인덱스를 의미, 여기서는 단순한 순서

In [ ]:
df2 = df2.set_index('Month')
df2.head()

In [ ]:
# index가 Month로 바뀌었으므로
df2.loc['08/01/2012']

In [ ]:
# 0번째
df2.iloc[0]

In [ ]:
df2 = df2.reset_index()

* multiple index

In [ ]:
df2 = df2.set_index(['Month', 'Stat'])
df2.head()

In [ ]:
df2.iloc[0]

In [ ]:
# multi index의 경우 ','로 구별하여 인덱싱 가능
df2.loc['08/01/2012', 'MatchesPlayed']

* grouping with multi index

In [ ]:
df2.head()

In [ ]:
stat_group = df2.groupby(level = 'Stat')
for name, group in stat_group:
    print name
    print group

In [ ]:
# group의 sum method를 이용
# 각 리그당 경기 수와 골을 나타냄
stat_group.sum()

In [ ]:
# 혹은 level을 이용하여 dataframe level에서 계산 가능
df2.sum(level = 'Stat')

In [ ]:
total_goal_df = stat_group.sum()
total_goal_df.head()

###  각 리그의 경기당 평균골 구하기

In [ ]:
df.head()

In [ ]:
total_goal_df.loc['GoalsScored'] / total_goal_df.loc['MatchesPlayed']

* 평균 골 Series 생성

In [ ]:
goals_per_game_df = total_goal_df.loc['GoalsScored'] / total_goal_df.loc['MatchesPlayed']
goals_per_game_df

* dataframe으로 변경

In [ ]:
pd.DataFrame(goals_per_game_df)

* Transpose하여 row, column 변경

In [ ]:
goals_per_game_df = pd.DataFrame(goals_per_game_df).T
goals_per_game_df

* mapping dictionary로 인덱스 값 변경

In [ ]:
goals_per_game_df = goals_per_game_df \
                        .rename(index = \
                                {0 : 'goals per game'})
goals_per_game_df

* 새로운 row로 추가

In [ ]:
total_goal_df.append(goals_per_game_df)

* 소수점 2자리까지 출력하도록 설정

In [ ]:
pd.options.display.float_format='{:.2f}'.format
total_goal_df.append(goals_per_game_df)

### Formating
  - https://pyformat.info/ 참고

### aggregate method

In [ ]:
print stat_group
for name, group in stat_group:
    print name
    print group

In [ ]:
stat_group.count()

In [ ]:
stat_group.aggregate(len)

In [ ]:
stat_group.aggregate(np.sum)

* multiple function 적용
  - 복수개의 column 레벨로 데이터 생성

In [ ]:
stat_group.aggregate([np.sum, np.mean, np.size])

### transform 함수

* 가상 sales data 실습
 - excel로 구성된 가상의 판매 데이터 이용하여 transformation 실습
 - excel 데이터를 읽기 위해서 xlrd 모듈이 필요
   - pip install xlrd

In [ ]:
import pandas as pd
df = pd.read_excel("sales_transactions.xlsx")

In [ ]:
df.head(13)

In [ ]:
df.groupby('order')["ext price"].sum()

* 위의 정보를 다시 원래의 frame에 합치려면?
 - 아래와 같이 다시 merge 하여 합칠 수도 있음
 - 가능은 하지만, 더 쉽게 transform으로 가능

In [ ]:
order_total = df.groupby('order')["ext price"].sum().rename("total").reset_index()
order_total

In [ ]:
result = df.merge(order_total, on='order')
result["portion"] = result["ext price"] / result["total"]

In [ ]:
result

* **transform 함수 사용!**
 - group by 후 transform을 사용하면 원본의 row를 유지한 상태에서 transform에 전달된 함수를 호출하여 aggregation

In [ ]:
df.groupby('order')["ext price"].transform(np.sum)

In [ ]:
df['total'] = df.groupby('order')["ext price"].transform(np.sum)
df["portion"] = df["ext price"] / df["total"]

In [ ]:
df

###  pivot 

In [ ]:
plant_df = pd.read_csv('./data/PlantGrowth.csv')
plant_df.head(30)

In [ ]:
plant_df
# control(ctrl)하여 실험한 값과, 두 개의 다른 조건(tr1, tr2)을 주었을 때의 실험에 대한 결과 
# data가 stacked 됨. (말 그대로 쌓여있음)
# 데이터 분석하기 편리하지 않음

In [ ]:
# 아래와 같이 filtering하여 각각의 결과를 뽑을 수 있으나, pivot, unstack을 이용하여 분석하기 용이하도록 변형 가능
plant_df[plant_df['group'] == 'ctrl']

In [ ]:
# dataframe reshaping
# 각각 index, columns, values로 명시하여 data frame의 shape을 변경 가능
plant_df.pivot(index = 'observation', 
               columns = 'group', 
               values = 'weight')

In [ ]:
# pivot_table 으로도 동일 결과 가능
pd.pivot_table(plant_df, values = 'weight', index = 'observation', columns='group')

* pivot, pivot_table의 차이점
 - pivot_table 
   - 어떤 값으로 채울지 모르는 경우 aggfunc 사용하여 채움


In [ ]:
pd.pivot_table(plant_df, values = 'weight', columns='group')

In [ ]:
pd.pivot_table(plant_df, values = 'weight', columns='observation')

In [ ]:
from collections import OrderedDict

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = pd.DataFrame(table)

In [ ]:
df

In [ ]:
df.pivot(index='Item', columns='CType', values='USD')

In [ ]:
df.pivot(index='Item', columns='CType')

In [ ]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))

df = pd.DataFrame(table)

df

In [ ]:
# Item0과 Gold에 대해서 1, 3중 어떤 것을 채워야 할지 알 수 없음
df.pivot(index='Item', columns='CType', values='USD')

In [ ]:
df.pivot_table(index='Item', 
               columns='CType', 
               values='USD',
              aggfunc = np.sum)

###  stack & unstack
 - stack : inner-most column -> inner-most index
 - unstack : inner-most index -> inner-most column
 
 - 둘은 역의 관계에 있음

In [ ]:
plant_df

In [ ]:
stacked = plant_df.set_index(['group', 'observation'])
stacked

In [ ]:
# 가장 바깥 레벨의 row -> column으로 이동
stacked.unstack()

In [ ]:
# 레벨 0의 row -> column으로 이동
stacked.unstack(0)

# 레벨의 순서가 아닌, 네임으로도 명시 가능
# stacked.unstack(level = 'group')

In [ ]:
stacked.unstack(0).stack()

### Concat

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [3]:
df1

,key1,value1
0,0,1.099095
1,1,0.046748
2,2,1.605853
3,3,0.177503
4,4,-1.390213
5,5,-0.286777
6,6,-0.119630
7,7,-0.220078
8,8,0.837083
9,9,-0.433428


In [4]:
df2 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [5]:
df2

,key1,value1
0,0,-1.336782
1,1,1.181736
2,2,-0.227069
3,3,0.776561
4,4,-0.738119
5,5,1.866567
6,6,-0.384694
7,7,0.656592
8,8,0.301558
9,9,-1.011377


In [6]:
# axis=0 기본적으로 row를 따라 합침
pd.concat([df1, df2])

,key1,value1
0,0,1.099095
1,1,0.046748
2,2,1.605853
3,3,0.177503
4,4,-1.390213
5,5,-0.286777
6,6,-0.119630
7,7,-0.220078
8,8,0.837083
9,9,-0.433428


In [7]:
# axis=1로 주어 column을 따라 합치는 것도 가능
pd.concat([df1, df2], axis=1)

,key1,value1,key1,value1
0,0,1.099095,0,-1.336782
1,1,0.046748,1,1.181736
2,2,1.605853,2,-0.227069
3,3,0.177503,3,0.776561
4,4,-1.390213,4,-0.738119
5,5,-0.286777,5,1.866567
6,6,-0.119630,6,-0.384694
7,7,-0.220078,7,0.656592
8,8,0.837083,8,0.301558
9,9,-0.433428,9,-1.011377


In [8]:
# axis=0 기본적으로 row를 따라 합침
# 이때 기존의 인덱스를 무시하려면 ignore_index 파라미터를 True로 변경
pd.concat([df1, df2], ignore_index=True)

,key1,value1
0,0,1.099095
1,1,0.046748
2,2,1.605853
3,3,0.177503
4,4,-1.390213
5,5,-0.286777
6,6,-0.119630
7,7,-0.220078
8,8,0.837083
9,9,-0.433428


* column 네임이 다를 경우

In [9]:
df3 = pd.DataFrame({'key2' : np.arange(10), 'value2' : np.random.randn(10)})

In [10]:
# 없는 컬럼에 대해서는 nan으로 채움
pd.concat([df1, df3], ignore_index=True)

,key1,key2,value1,value2
0,0.0,NaN,1.099095,NaN
1,1.0,NaN,0.046748,NaN
2,2.0,NaN,1.605853,NaN
3,3.0,NaN,0.177503,NaN
4,4.0,NaN,-1.390213,NaN
5,5.0,NaN,-0.286777,NaN
6,6.0,NaN,-0.119630,NaN
7,7.0,NaN,-0.220078,NaN
8,8.0,NaN,0.837083,NaN
9,9.0,NaN,-0.433428,NaN


* index를 설정하고 concat

In [11]:
df11 = df1.set_index('key1')

In [12]:
df22 = df2.set_index('key1')

In [13]:
pd.concat([df11, df22])

,value1
key1,
0,1.099095
1,0.046748
2,1.605853
3,0.177503
4,-1.390213
5,-0.286777
6,-0.119630
7,-0.220078
8,0.837083


In [14]:
pd.concat([df11, df22], axis=1)

,value1,value1
key1,,
0,1.099095,-1.336782
1,0.046748,1.181736
2,1.605853,-0.227069
3,0.177503,0.776561
4,-1.390213,-0.738119
5,-0.286777,1.866567
6,-0.119630,-0.384694
7,-0.220078,0.656592
8,0.837083,0.301558


### Merge & Join

In [15]:
df1 = pd.DataFrame({'key1' : np.arange(10), 
                    'key2' : ['LA', 'LA', 'SF', 'SF', 'SF', 'NY', 'NY', 'LV', 'LV', 'SF'], 
                    'value1' : np.random.randn(10)})

In [16]:
df1

,key1,key2,value1
0,0,LA,-0.790678
1,1,LA,-0.130006
2,2,SF,0.202797
3,3,SF,-0.394682
4,4,SF,0.772220
5,5,NY,-0.836091
6,6,NY,-0.407691
7,7,LV,-1.713573
8,8,LV,1.309043
9,9,SF,-0.339855


In [17]:
df2 = pd.DataFrame({'key1' : [1, 1, 2, 2, 3, 3, 4, 4, 5, 5], 
                    'key2' : ['LA', 'LA', 'LA', 'KC', 'KC', 'NY', 'NY', 'NY', 'SF', 'SF'], 
                    'value1' : np.random.randn(10)})

In [18]:
df2

,key1,key2,value1
0,1,LA,0.070621
1,1,LA,2.582324
2,2,LA,-1.039770
3,2,KC,-0.526453
4,3,KC,-1.183609
5,3,NY,0.502367
6,4,NY,0.012478
7,4,NY,0.334135
8,5,SF,0.322807
9,5,SF,1.489749


* on
 - join 하고자 하는 대상이 되는 column 명시

In [19]:
pd.merge(df1, df2, on='key1')

,key1,key2_x,value1_x,key2_y,value1_y
0,1,LA,-0.130006,LA,0.070621
1,1,LA,-0.130006,LA,2.582324
2,2,SF,0.202797,LA,-1.039770
3,2,SF,0.202797,KC,-0.526453
4,3,SF,-0.394682,KC,-1.183609
5,3,SF,-0.394682,NY,0.502367
6,4,SF,0.772220,NY,0.012478
7,4,SF,0.772220,NY,0.334135
8,5,NY,-0.836091,SF,0.322807
9,5,NY,-0.836091,SF,1.489749


In [20]:
pd.merge(df1, df2, on='key2')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.790678,1,0.070621
1,0,LA,-0.790678,1,2.582324
2,0,LA,-0.790678,2,-1.039770
3,1,LA,-0.130006,1,0.070621
4,1,LA,-0.130006,1,2.582324
5,1,LA,-0.130006,2,-1.039770
6,2,SF,0.202797,5,0.322807
7,2,SF,0.202797,5,1.489749
8,3,SF,-0.394682,5,0.322807
9,3,SF,-0.394682,5,1.489749


* how
 - SQL의 join과 같이 동작
 - inner : 기본 값, 일치 하는 값들로만 조인
 - left : left outer join과 같이 동작
 - right : right outer join과 같이 동작
 - outer : full outer join과 같이 동작, 즉 left + right

In [21]:
pd.merge(df1, df2, on='key2', how='left')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.790678,1.0,0.070621
1,0,LA,-0.790678,1.0,2.582324
2,0,LA,-0.790678,2.0,-1.039770
3,1,LA,-0.130006,1.0,0.070621
4,1,LA,-0.130006,1.0,2.582324
5,1,LA,-0.130006,2.0,-1.039770
6,2,SF,0.202797,5.0,0.322807
7,2,SF,0.202797,5.0,1.489749
8,3,SF,-0.394682,5.0,0.322807
9,3,SF,-0.394682,5.0,1.489749


In [22]:
pd.merge(df1, df2, on='key2', how='right')

,key1_x,key2,value1_x,key1_y,value1_y
0,0.0,LA,-0.790678,1,0.070621
1,1.0,LA,-0.130006,1,0.070621
2,0.0,LA,-0.790678,1,2.582324
3,1.0,LA,-0.130006,1,2.582324
4,0.0,LA,-0.790678,2,-1.039770
5,1.0,LA,-0.130006,2,-1.039770
6,2.0,SF,0.202797,5,0.322807
7,3.0,SF,-0.394682,5,0.322807
8,4.0,SF,0.772220,5,0.322807
9,9.0,SF,-0.339855,5,0.322807


* multi key 조인
 - 리스트 형식으로 전달

In [23]:
pd.merge(df1, df2, on=['key1', 'key2'])

,key1,key2,value1_x,value1_y
0,1,LA,-0.130006,0.070621
1,1,LA,-0.130006,2.582324


In [24]:
pd.merge(df1, df2, on=['key1', 'key2'], how='left')

,key1,key2,value1_x,value1_y
0,0,LA,-0.790678,NaN
1,1,LA,-0.130006,0.070621
2,1,LA,-0.130006,2.582324
3,2,SF,0.202797,NaN
4,3,SF,-0.394682,NaN
5,4,SF,0.772220,NaN
6,5,NY,-0.836091,NaN
7,6,NY,-0.407691,NaN
8,7,LV,-1.713573,NaN
9,8,LV,1.309043,NaN


In [25]:
# 나머지 column의 이름이 겹칠 경우 suffix 지정 가능
pd.merge(df1, df2, on=['key1', 'key2'], how='left', 
         suffixes=('_left', '_right'))

,key1,key2,value1_left,value1_right
0,0,LA,-0.790678,NaN
1,1,LA,-0.130006,0.070621
2,1,LA,-0.130006,2.582324
3,2,SF,0.202797,NaN
4,3,SF,-0.394682,NaN
5,4,SF,0.772220,NaN
6,5,NY,-0.836091,NaN
7,6,NY,-0.407691,NaN
8,7,LV,-1.713573,NaN
9,8,LV,1.309043,NaN


* index가 있는 경우 merge
 - left_index : 왼쪽 프레임에서 인덱스를 사용할 경우 True 명시
 - right_index : 오른쪽 프레임에서 인덱스를 사용할 경우 True 명시

In [26]:
df11 = df1.set_index('key1')

In [27]:
df11

,key2,value1
key1,,
0,LA,-0.790678
1,LA,-0.130006
2,SF,0.202797
3,SF,-0.394682
4,SF,0.772220
5,NY,-0.836091
6,NY,-0.407691
7,LV,-1.713573
8,LV,1.309043


In [28]:
df22 = df2.set_index('key1')

In [29]:
pd.merge(df11, df22, left_index=True, right_index=True)

,key2_x,value1_x,key2_y,value1_y
key1,,,,
1,LA,-0.130006,LA,0.070621
1,LA,-0.130006,LA,2.582324
2,SF,0.202797,LA,-1.039770
2,SF,0.202797,KC,-0.526453
3,SF,-0.394682,KC,-1.183609
3,SF,-0.394682,NY,0.502367
4,SF,0.772220,NY,0.012478
4,SF,0.772220,NY,0.334135
5,NY,-0.836091,SF,0.322807
